<!-- -*- mode: markdown; coding: utf-8; fill-column: 60; ispell-dictionary: "english" -*- -->

<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<link rel="stylesheet" href="style.css">


# EDAF75 - lab 2: Testing the database

As usual we have to tell Jupyter to allow SQL:

In [1]:
%load_ext sql

And then we import our movie database

In [2]:
%sql sqlite:///movies.sqlite

We want to make sure that SQLite3 really checks our foreign
key constraints -- to do that, we run:

In [3]:
%%sql
PRAGMA foreign_keys=ON;

 * sqlite:///movies.sqlite
Done.


[]

## SQL statements to review at the lab session

Write SQL code for the following tasks:

+ Show the names of all movies.

In [36]:
%%sql
select name
from movies

 * sqlite:///movies.sqlite
Done.


name
The Gentlemen
The Godfather
The Godfather 2
The Godfather 3


+ Show the performance dates for one of the movies.

In [37]:
%%sql
select name, date, start_time as "Start time"
from movies
join screenings
on movies.name = screenings.movie_name
where name = "The Godfather"
order by date,start_time

 * sqlite:///movies.sqlite
Done.


name,date,Start time
The Godfather,2022-02-01,18:00
The Godfather,2022-02-01,19:30
The Godfather,2022-02-01,21:15


+ Show all data concerning performances at a given theatere
  on a given date.

In [4]:
%%sql
select *
from screenings as s
join theatres as t 
on s.theatre_name = t.name
where name = "Victoria" and date = "2022-02-01"

 * sqlite:///movies.sqlite
Done.


theatre_name,start_time,date,movie_name,production_year,name,capacity
Victoria,18:00,2022-02-01,The Godfather,1972,Victoria,220
Victoria,21:15,2022-02-01,The Godfather,1972,Victoria,220


+ List all customers

In [39]:
%%sql
select *
from customers

 * sqlite:///movies.sqlite
Done.


user_name,full_name,password
kingen,Arvid Gramer,hejsan
fransan,Fanny Spindler Lagercrantzz,arvid<3
persimon,Simon Danielsson,cmdlinerules
chuppen,Charlie Nilsson,snartFårJagSpelaWoW
nelson11,Nils Romanus Myrberg,snartFårJagSpelaWoW


In [14]:
%%sql
select user_name, full_name, movie_name, date, start_time, ticket_number
from customers
join tickets
using (user_name)
join screenings
where (start_time, date)

 * sqlite:///movies.sqlite
(sqlite3.OperationalError) ambiguous column name: date
[SQL: select user_name, full_name, movie_name, date, start_time, ticket_number
from customers
join tickets
using (user_name)
join screenings
where (start_time, date)]
(Background on this error at: http://sqlalche.me/e/e3q8)


+ List all tickets

In [6]:
%%sql
select *
from tickets

 * sqlite:///movies.sqlite
Done.


ticket_number,start_time,date,theatre,user_name
002212,19:30,2022-02-01,Rigoletto,kingen
002213,19:30,2022-02-01,Rigoletto,fransan
001233,19:30,2022-02-01,Rigoletto,persimon
002214,19:30,2022-02-01,Rigoletto,chuppen
002241,19:30,2022-02-01,Rigoletto,nelson11


In [39]:
%%sql
with seats_left(date, start_time, theatre_name, nbr) as(
    select ti.date, ti.start_time, th.name, th.capacity - count(ti.ticket_number)
    from theatres as th
    left outer join screenings as s
    on s.theatre_name = th.name
    join tickets as ti
    on ti.theatre = th.name
)
select *
from seats_left


 * sqlite:///movies.sqlite
Done.


date,start_time,theatre_name,nbr
2022-02-01,19:30,Rigoletto,895


In [10]:
%sql sqlite:///movies_3.sqlite

In [18]:
%%sql
select * 
from theatres

   sqlite:///movies.sqlite
 * sqlite:///movies_3.sqlite
Done.


name,capacity
Rigoletto,900
Söder,150
Victoria,220
Astoria,450


In [78]:
%%sql
WITH seats_left(screening_id, remaining_tickets) as (
SELECT s.screening_id, th.capacity - count(ti.ticket_number)
FROM theatres AS th
LEFT outer join screenings AS s
ON s.theatre_name = th.name
JOIN tickets AS ti
ON ti.start_time  = s.start_time
GROUP BY s.start_time, s.date
)

SELECT screening_id, date, start_time, movie_name, production_year, theatre_name, remaining_tickets
FROM screenings
LEFT OUTER JOIN seats_left using (screening_id)

 * sqlite:///movies.sqlite
   sqlite:///movies_2.sqlite
   sqlite:///movies_3.sqlite
(sqlite3.OperationalError) cannot join using column screening_id - column not present in both tables
[SQL: WITH seats_left(screening_id, remaining_tickets) as (
SELECT s.screening_id, th.capacity - count(ti.ticket_number)
FROM theatres AS th
LEFT outer join screenings AS s
ON s.theatre_name = th.name
JOIN tickets AS ti
ON ti.start_time  = s.start_time
GROUP BY s.start_time, s.date
)

SELECT screening_id, date, start_time, movie_name, production_year, theatre_name, remaining_tickets
FROM screenings
LEFT OUTER JOIN seats_left using (screening_id)]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [77]:
%%sql
SELECT *
FROM tickets
JOIN screenings
using (date, start_time)

 * sqlite:///movies.sqlite
   sqlite:///movies_2.sqlite
   sqlite:///movies_3.sqlite
Done.


ticket_number,start_time,date,theatre,user_name,theatre_name,movie_name,production_year
002212,19:30,2022-02-01,Rigoletto,kingen,Rigoletto,The Godfather,1972
002213,19:30,2022-02-01,Rigoletto,fransan,Rigoletto,The Godfather,1972
001233,19:30,2022-02-01,Rigoletto,persimon,Rigoletto,The Godfather,1972
002214,19:30,2022-02-01,Rigoletto,chuppen,Rigoletto,The Godfather,1972
002241,19:30,2022-02-01,Rigoletto,nelson11,Rigoletto,The Godfather,1972


In [76]:
%%sql
SELECT *
FROM screenings

 * sqlite:///movies.sqlite
   sqlite:///movies_2.sqlite
   sqlite:///movies_3.sqlite
Done.


theatre_name,start_time,date,movie_name,production_year
Rigoletto,19:30,2022-02-01,The Godfather,1972
Söder,17:30,2022-02-02,The Gentlemen,2019
Victoria,18:00,2022-02-01,The Godfather,1972
Victoria,21:15,2022-02-01,The Godfather,1972


In [13]:
%%sql
INSERT
INTO     tickets(ticket_number, start_time, date, theatre, user_name)
VALUES   ('102212', '19:30', '2022-02-01', 'Rigoletto', 'kingen'),
         ('102213', '19:30', '2022-02-01', 'Rigoletto', 'fransan');

   sqlite:///movies.sqlite
 * sqlite:///movies_3.sqlite
2 rows affected.


[]

In [17]:
%%sql
SELECT *
FROM screenings

   sqlite:///movies.sqlite
 * sqlite:///movies_3.sqlite
Done.


theatre_name,start_time,date,movie_name,production_year,screening_id


In [14]:
%%sql
SELECT s.date, s.start_time, theatre, movie_name, production_year, count() as nbr_of_tickets
FROM tickets AS ti
JOIN screenings AS s
ON s.date = ti.date AND s.start_time = ti.start_time AND s.theatre_name = ti.theatre
GROUP BY user_name
HAVING user_name = "kingen"

   sqlite:///movies.sqlite
 * sqlite:///movies_3.sqlite
Done.


date,start_time,theatre,movie_name,production_year,nbr_of_tickets


## Things to try out yourselves (not reviewed at the lab session)

During lab 3 you'll write a REST service for the database
you just created, and for the server to work properly, the
things below should work.

If you have any questions about the material below, feel
free to ask them at the QA sessions (see Moodle for how to
sign up for them).


+ Create a new ticket to some performance (i.e., insert a
  new row in your table of tickets).

In [ ]:
%%sql


In newer versions of SQLite (since version 3.35, released
  in March 2021), and in
  [PostgreSQL](https://www.postgresql.org/docs/current/sql-insert.html),
  we can get any value generated during an insert using the
  `INSERT...-RETURNING` statement:

~~~{.sql}
INSERT
INTO       students
VALUES     ('Amy', 3.9, 1200)
RETURNING  s_id
~~~


which would return the generated `s_id` for the new
  student.

  If your SQLite version is older than 3.35, and you can't
  upgrade, you can instead use the following idea: each row
  in a SQLite3 table has a `rowid` attribute, it is a unique
  integer which essentially tells in which order the rows
  were inserted, and it's not displayed in queries unless we
  ask for it. SQLite3 also have a function,
  `last_insert_rowid()`, which returns the `rowid` of the
  last inserted row of a table, so we can see the `s_id` of
  the most recently inserted student with the following
  query:

~~~{.sql}
SELECT s_id
FROM   students
WHERE  rowid = last_insert_rowid();
~~~


Now, check what ticket number we got for the ticket we
  created above (it should be the same as the ticket id,
  which should be a `randomblob`):

In [5]:
%%sql
SELECT s.date, s.start_time, theatre, movie_name, production_year, count() as nbr_of_tickets
FROM tickets AS ti
JOIN screenings AS s
ON s.date = ti.date AND s.start_time = ti.start_time AND s.theatre_name = ti.theatre
GROUP BY user_name
HAVING user_name = "nelson11"

 * sqlite:///movies.sqlite
Done.


date,start_time,theatre,movie_name,production_year,nbr_of_tickets
2022-02-01,19:30,Rigoletto,The Godfather,1972,1


+ Try to insert two movie theaters with the same name (this
  should fail).

In [ ]:
%%sql


+ Try to insert a performance where the theater doesn’t
  exist in the database (this should fail).

In [ ]:
%%sql


+ Create a ticket where either the user or the performance
  doesn’t exist (this should fail).

In [ ]:
%%sql


In [7]:
import sqlite3

In [16]:
conn = sqlite3.connect('localhost:7007')
cursor = conn.cursor()

# Execute an SQL query
cursor.execute("""SELECT name FROM sqlite_master  WHERE type='table';""")
print(cursor.fetchall())

[]
